# Juntando tabelas com SQL (JOIN)


## O que é um JOIN?
JOIN é a operação que permite **cruzar dados de duas ou mais tabelas** que possuem uma coluna em comum.

**Exemplo real:**
- A tabela `clientes` tem o nome e cidade da pessoa
- A tabela `compras` tem o valor que ela gastou
- Queremos **juntar essas informações**

## INNER JOIN vs LEFT JOIN
- `INNER JOIN` = só mostra quem está nas duas tabelas
- `LEFT JOIN` = mostra todos da tabela da esquerda, mesmo que não tenham correspondência na direita

**Analogia:**
Imagine uma lista de alunos e uma lista de quem entregou o trabalho:
- `INNER JOIN` → só mostra quem está nas duas listas
- `LEFT JOIN` → mostra todos os alunos, mesmo os que não entregaram

## Criando os DataFrames para simular duas tabelas

In [1]:
import pandas as pd
import sqlite3

# Dados dos clientes
clientes = pd.DataFrame({
    'id_cliente': [1, 2, 3, 4],
    'nome': ['Ana', 'Carlos', 'Joana', 'Marcelo'],
    'cidade': ['SP', 'RJ', 'MG', 'RS']
})

# Dados das compras
compras = pd.DataFrame({
    'id_compra': [101, 102, 103, 104],
    'id_cliente': [1, 1, 2, 3],
    'valor': [250.0, 180.0, 90.0, 300.0]
})

## Enviando para o SQLite

In [2]:
conn = sqlite3.connect(":memory:")
clientes.to_sql("clientes", conn, index=False, if_exists="replace")
compras.to_sql("compras", conn, index=False, if_exists="replace")

4

## Fazendo um INNER JOIN – clientes que fizeram compras

In [8]:
resultados_inner = pd.read_sql_query('''
SELECT c.nome, c.cidade, SUM(co.valor)
FROM clientes c
INNER JOIN compras co ON c.id_cliente = co.id_cliente
Group by c.id_cliente
''', conn)

In [5]:
clientes

,id_cliente,nome,cidade
0,1,Ana,SP
1,2,Carlos,RJ
2,3,Joana,MG
3,4,Marcelo,RS


In [6]:
compras

,id_compra,id_cliente,valor
0,101,1,250.0
1,102,1,180.0
2,103,2,90.0
3,104,3,300.0


In [9]:
resultados_inner.head()

,nome,cidade,SUM(co.valor)
0,Ana,SP,430.0
1,Carlos,RJ,90.0
2,Joana,MG,300.0


## Fazendo um LEFT JOIN – todos os clientes, mesmo os que não compraram

In [10]:
resultados_left = pd.read_sql_query('''
SELECT c.nome, c.cidade, co.valor
FROM clientes c
LEFT JOIN compras co ON c.id_cliente = co.id_cliente
''', conn)

In [11]:
resultados_left

,nome,cidade,valor
0,Ana,SP,180.0
1,Ana,SP,250.0
2,Carlos,RJ,90.0
3,Joana,MG,300.0
4,Marcelo,RS,NaN


## Conclusão
- JOINs permitem **cruzar informações complementares**
- `INNER JOIN`: traz apenas as correspondências
- `LEFT JOIN`: traz todos da esquerda, mesmo sem relação
- Essencial para qualquer análise de dados em bases reais